In [1]:
from utils.infer import *
import pandas as pd
from  utils.eval import *
from utils.chat import *
from utils.data import *

%reload_ext autoreload
%autoreload 2

/Users/paul/anaconda3/envs/ClinicalNotes/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Combining inferences

In [2]:
inference_paths = "../data/inference"
output_path = "../data/combined_inferences.jsonl"

combined = combine(inference_paths, output_path)

### Building evaluation inputs

In [4]:
build_evaluation_inputs(output_path, models = ['direct-gpt', 'meditron-7b-summarizer', 'meditron-13b-summarizer'])

### Evaluation GPT 3 direct

In [5]:
gpt_3_eval_res = note_evaluation('direct-gpt')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [6]:
display(gpt_3_eval_res)

,idx,gold,pred,model_name,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum
0,60564,A 29-year-old lady presented to the surgical o...,Patient presented to the surgical department w...,direct-gpt,0.193591,0.875239,9,0.867925,0.696682,0.528302,0.528302
1,87134,A 32-year-old male presented to the accident a...,Patient presented to the emergency department ...,direct-gpt,0.384111,0.855199,8,0.536723,0.295455,0.451977,0.451977
2,54758,A 4 year-old previously healthy girl was trans...,"Patient presented with complaints of weakness,...",direct-gpt,0.092334,0.836584,7,0.493671,0.280255,0.398734,0.398734
3,62070,A 53-year-old male with a history significant ...,Patient presented with a chief complaint of ab...,direct-gpt,0.358175,0.898979,9,0.719388,0.538462,0.642857,0.642857
4,183934,A 44-year-old female presented to the emergenc...,Patient presented to the emergency room with s...,direct-gpt,0.240081,0.872304,9,0.685851,0.525301,0.637890,0.637890
...,...,...,...,...,...,...,...,...,...,...,...
995,126484,A 23-year-old man with a healthy appearance (1...,Patient presented to the clinic with persisten...,direct-gpt,0.318324,0.867252,7,0.481481,0.186567,0.377778,0.377778
996,72811,An 82-year-old man presented to us for right e...,Patient presented with a chief complaint of ri...,direct-gpt,0.227259,0.878446,9,0.417445,0.263323,0.367601,0.367601
997,18020,A 61-year-old male with a history of inotrope-...,Patient presented with symptoms of heart issue...,direct-gpt,0.351929,0.872541,8,0.542056,0.357367,0.485981,0.485981
998,122718,A 33-year-old woman who was diagnosed with thy...,Patient presented for a follow-up examination ...,direct-gpt,0.241049,0.838174,9,0.652819,0.429851,0.599407,0.599407


In [7]:
print(gpt_3_eval_res[['bleu','bert','gpt_score'] + ROUGE_SUB_SCORES].mean())

bleu         0.250703
bert         0.876228
gpt_score    8.196000
rouge1       0.611542
rouge2       0.408162
rougeL       0.531128
rougeLsum    0.531128
dtype: float64


### Evaluation meditron-7b-summarizer

In [22]:
summarizer7_eval_by_sample , summarizer7_eval_by_key = summary_evaluation('meditron-7b-summarizer')

Running summary evaluation with score types: ['bleu', 'rouge', 'bert', 'gpt_score']
Loading data...
Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'flatten and match dicts': 'done', 'clean dicts and counts': 'done', 'summary_statistics': 'done', 'scores': 'done', 'pairs_idx': 'done', 'eval_by_sample': 'done', 'eval_by_key': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [23]:
display(summarizer7_eval_by_sample)
display(summarizer7_eval_by_key)

,idx,gold,pred,mean_bleu,mean_bert,mean_gpt_score,mean_rouge1,mean_rouge2,mean_rougeL,mean_rougeLsum,missing_keys_count,extra_keys_count,common_none_count,gold_none_count,pred_none_count,common_non_none,all_keys_count
0,127042,{'visit motivation': 'Severe pain and restrict...,{'visit motivation': 'Pain in the right should...,0.024697,0.781193,7.000000,0.300449,0.091925,0.282489,0.282489,7,0,30,3,2,23,65
1,131562,{'visit motivation': 'Abdominal mass that had ...,"{'visit motivation': 'Abdominal mass, loss of ...",0.154609,0.869353,6.888889,0.553576,0.334366,0.537042,0.537042,3,0,38,0,2,18,61
2,141471,{'visit motivation': 'History of flank pain on...,{'visit motivation': 'Right-sided flank pain a...,0.057203,0.835415,5.600000,0.383232,0.206255,0.374036,0.374036,28,0,16,4,4,15,67
3,117562,{'visit motivation': 'Subacute progressive bil...,{'visit motivation': 'Progressive bilateral le...,0.226453,0.888196,7.468750,0.547230,0.420537,0.528320,0.528320,6,0,29,3,5,32,75
4,55332,{'visit motivation': 'Dyspnea since three mont...,{'visit motivation': 'Persistent dyspnea for t...,0.136109,0.893432,7.600000,0.510297,0.347374,0.510297,0.510297,34,0,28,1,3,25,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,93078,{'visit motivation': 'Anterior midline neck ma...,{'visit motivation': 'Anterior midline neck ma...,0.193582,0.898657,7.450000,0.657176,0.503265,0.638362,0.638362,25,0,23,3,2,20,73
996,38574,{'visit motivation': 'unilateral left knee pai...,"{'visit motivation': 'Left knee pain', 'admiss...",0.171704,0.901672,6.312500,0.575768,0.451841,0.563573,0.563573,16,0,19,5,6,16,62
997,65166,"{'visit motivation': 'Painful, swollen right k...",{'visit motivation': 'Painful and swollen righ...,0.210072,0.888840,7.437500,0.618466,0.490061,0.609265,0.609265,0,0,31,2,2,32,67
998,157147,{'visit motivation': 'Acute cerebral air embol...,{'visit motivation': 'Acute onset of unrespons...,0.172424,0.837329,4.722222,0.415853,0.344101,0.415853,0.415853,15,0,34,2,7,18,76


,keys,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,gold_none_prop,pred_none_prop,missing_keys_prop
0,admission/0/care center details,0.270086,0.918273,7.611111,0.735385,0.650588,0.726991,0.726991,0.048,0.153,0.012
1,admission/0/date,0.231097,0.919139,7.827957,0.761162,0.608599,0.754994,0.754994,0.020,0.095,0.012
2,admission/0/duration,0.116882,0.927714,8.325843,0.762572,0.690028,0.762572,0.762572,0.021,0.049,0.012
3,admission/0/reason,0.262550,0.884549,7.142216,0.552512,0.429219,0.531419,0.531419,0.038,0.069,0.012
4,admission/1/care center details,0.082227,0.941237,8.200000,0.795455,0.666667,0.795455,0.795455,0.001,0.003,0.010
...,...,...,...,...,...,...,...,...,...,...,...
223,treatments/5/reaction to treatment,0.000000,0.926654,8.000000,0.533333,0.461538,0.533333,0.533333,0.000,0.000,0.016
224,treatments/5/reason for taking,0.333333,0.809110,4.666667,0.416667,0.333333,0.416667,0.416667,0.000,0.000,0.016
225,treatments/5/related condition,0.000000,0.833170,5.500000,0.500000,0.000000,0.500000,0.500000,0.000,0.000,0.016
226,treatments/5/time,0.000000,0.659807,1.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.016


### Evaluation meditron-13b-summarizer

In [24]:
summarizer13_eval_by_sample , summarizer13_eval_by_key = summary_evaluation('meditron-13b-summarizer')

Running summary evaluation with score types: ['bleu', 'rouge', 'bert', 'gpt_score']
Loading data...
Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'flatten and match dicts': 'done', 'clean dicts and counts': 'done', 'summary_statistics': 'done', 'scores': 'done', 'pairs_idx': 'done', 'eval_by_sample': 'done', 'eval_by_key': 'tbd', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [26]:
display(summarizer13_eval_by_sample)
display(summarizer13_eval_by_key)

,idx,gold,pred,mean_bleu,mean_bert,mean_gpt_score,mean_rouge1,mean_rouge2,mean_rougeL,mean_rougeLsum,missing_keys_count,extra_keys_count,common_none_count,gold_none_count,pred_none_count,common_non_none,all_keys_count
0,127042,{'visit motivation': 'Severe pain and restrict...,{'visit motivation': 'Pain and limited shoulde...,0.052057,0.809359,4.545455,0.355286,0.167743,0.323017,0.323017,7,0,30,3,3,22,65
1,131562,{'visit motivation': 'Abdominal mass that had ...,"{'visit motivation': 'Abdominal mass', 'admiss...",0.265478,0.878911,7.150000,0.559249,0.387131,0.554587,0.554587,0,0,38,0,3,20,61
2,141471,{'visit motivation': 'History of flank pain on...,{'visit motivation': 'Right flank pain and dys...,0.042897,0.838990,4.800000,0.417792,0.208620,0.405431,0.405431,28,0,16,4,4,15,67
3,117562,{'visit motivation': 'Subacute progressive bil...,{'visit motivation': 'Progressive bilateral le...,0.242662,0.926535,8.242424,0.717230,0.571073,0.708446,0.708446,10,0,25,3,4,33,75
4,55332,{'visit motivation': 'Dyspnea since three mont...,{'visit motivation': 'Persistent dyspnea for t...,0.138540,0.908767,7.375000,0.617728,0.404067,0.617728,0.617728,37,0,26,1,3,24,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,93078,{'visit motivation': 'Anterior midline neck ma...,{'visit motivation': 'Anterior midline neck ma...,0.229592,0.917562,8.047619,0.723265,0.569844,0.691038,0.691038,16,0,28,3,5,21,73
996,38574,{'visit motivation': 'unilateral left knee pai...,{'visit motivation': 'Moderate left knee pain'...,0.151924,0.915704,7.294118,0.598032,0.478081,0.589423,0.589423,16,0,19,5,5,17,62
997,65166,"{'visit motivation': 'Painful, swollen right k...",{'visit motivation': 'Painful and swollen righ...,0.220954,0.888004,7.375000,0.635147,0.511343,0.633308,0.633308,0,0,33,0,2,32,67
998,157147,{'visit motivation': 'Acute cerebral air embol...,{'visit motivation': 'Acute onset of aphasia a...,0.155020,0.840372,4.782609,0.433216,0.328084,0.416550,0.416550,9,0,36,0,8,23,76


,keys,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,gold_none_prop,pred_none_prop,missing_keys_prop
0,admission/0/care center details,0.264811,0.920767,7.886364,0.733560,0.643519,0.728607,0.728607,0.050,0.167,0.014
1,admission/0/date,0.222130,0.915933,7.511111,0.744379,0.559630,0.735311,0.735311,0.019,0.097,0.014
2,admission/0/duration,0.121161,0.941716,8.843373,0.808594,0.726863,0.806249,0.806249,0.019,0.054,0.014
3,admission/0/reason,0.255464,0.885055,6.565868,0.553590,0.427894,0.533150,0.533150,0.050,0.067,0.014
4,admission/1/care center details,0.160790,0.945127,8.600000,0.815455,0.777778,0.815455,0.815455,0.001,0.003,0.009
...,...,...,...,...,...,...,...,...,...,...,...
221,treatments/4/time,0.000000,0.815932,3.000000,0.272727,0.200000,0.272727,0.272727,0.000,0.000,0.026
222,treatments/5/name,0.000000,0.933010,5.500000,0.750000,0.500000,0.750000,0.750000,0.000,0.000,0.017
223,treatments/5/reason for taking,0.762368,0.993797,9.500000,0.900000,0.807692,0.900000,0.900000,0.000,0.000,0.017
224,treatments/5/related condition,0.000000,0.950773,9.000000,0.857143,0.400000,0.571429,0.571429,0.000,0.000,0.017
